# Team22 parser
### Забирает с персональных страничек ФИО, телефоны и имэйлы и возвращает структурированный словарь.

In [1]:
import re
import requests
from bs4 import BeautifulSoup


class Team22Parser:
    """
    Класс для получения ФИО, номеров телефонов и адресов электронной почты с сайта
    http://absda.ru:15000/team22/users/base/team.html
    """
    SRC_URL = 'http://absda.ru:15000/team22/users/base/team.html'
    BASE_URL = 'http://absda.ru:15000'

    @staticmethod
    def get_soup(url: str) -> BeautifulSoup:
        """
        Метод делает из веб-страницы и возвращает объект BeautifulSoup для дальнейшего разбора.
        """
        return BeautifulSoup(requests.get(url).content, 'lxml')

    @staticmethod
    def check_text_for_name(input_string: str) -> bool:
        """
        Метод проверяет, может ли строка быть ФИО.
        """
        words = input_string.split()
        count = len([word for word in words if word.istitle()])

        if len(words) == 2 or len(words) == 3:
            if count == len(words):
                return True
            else:
                return False
        else:
            return False

    @classmethod
    def get_data(cls) -> dict:
        """
        Метод возвращает словарь с полученными данными.
        """
        data_dict = {item.text: {'link': cls.BASE_URL + item.get('href')} for item in
                     cls.get_soup(cls.SRC_URL).find_all('a')}

        for name, value in data_dict.items():
            html = str(cls.get_soup(value['link']))

            text_list = re.findall(r'>(.*)<', html)
            for text in text_list:
                if cls.check_text_for_name(text):
                    data_dict[name]['name'] = text
                    break

            phone_pattern = re.compile(r'\+?\d[( -]?\d{3}[) -]?\d{3}[ -]?\d{2}[ -]?\d{2}')
            phones = re.findall(phone_pattern, html)
            clear_phones = set()
            for n, phone in enumerate(phones):
                if phone[0] == '8' or phone[:2] == '+7':
                    clear_phones.add('+7' + ''.join(num for num in phone if num.isdigit())[-10:])
            if clear_phones:
                data_dict[name]['phones'] = list(clear_phones)

            email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
            emails = set(re.findall(email_pattern, html))
            if emails:
                data_dict[name]['emails'] = list(emails)

        return data_dict


print(*Team22Parser.get_data().items(), sep='\n')

('Andromary', {'link': 'http://absda.ru:15000/team22/users/andromary/index.html'})
('AndVlIv', {'link': 'http://absda.ru:15000/team22/users/andvliv/my_page.html', 'name': 'Andrey Ivanchenko'})
('AnnaMargolina', {'link': 'http://absda.ru:15000/team22/users/annamargolina/cv.html'})
('anton-kuvalda', {'link': 'http://absda.ru:15000/team22/users/anton-kuvalda/Anton_kuvalda.Resume4.html', 'name': 'Резюме Непочатый Антон', 'emails': ['kuvaldabikes@gmail.com']})
('arisova', {'link': 'http://absda.ru:15000/team22/users/aaarisova/ArisovaA_zarplata_ru.html', 'emails': ['anna.arisova@example.com']})
('re-gi-na', {'link': 'http://absda.ru:15000/team22/users/re-gi-na/about_me.html', 'emails': ['hogo88@mail.ru']})
('RomanTarasov91', {'link': 'http://absda.ru:15000/team22/users/romantarasov91/aboutmeee.html', 'name': 'Тарасов Роман Сергеевич', 'phones': ['+79264790084'], 'emails': ['kenn-nni@yandex.ru', 'RomanTarasov91@yandex.ru']})
('Slash3086', {'link': 'http://absda.ru:15000/team22/users/slash3086